In [1]:
%load_ext autoreload
%autoreload 2

import locale 

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from IPython.display import display

import iccas as ic
import util

%matplotlib inline
plt.style.use('seaborn')
mpl.rcParams.update({
    'axes.titlesize': 16,
    'axes.titley': 1.05,
    'axes.labelsize': 14,
})

locale.setlocale(locale.LC_ALL, 'Italian');

data = ic.get()

## Dati inconsistenti

Dato che l'ISS riporta il **numero di casi e decessi totali a partire dall'inizio dell'epidemia**,
le serie temporali dovrebbero essere non decrescenti per ogni fascia d'età. 
Tuttavia, non lo sono. Per esempio, la tabella dei decessi sotto mostra come alcune 
serie temporali vadano su per poi riscendere di valore.

In [2]:
d = data.deaths.loc['2020-05':]
d['Totale'] = d.sum(axis=1)
(d.drop(columns='unknown').style
     .background_gradient()
     .set_caption("Decessi dall'inizio della pandemia")
)

age_group,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,>=90,Totale
date,,,,,,,,,,,
2020-05-07 16:00:00,3,0,9,54,246,993,2976,7849,11395,4430,27955
2020-05-14 16:00:00,3,0,12,59,258,1063,3127,8221,12104,4844,29692
2020-05-20 16:00:00,4,0,14,61,268,1101,3219,8447,12691,5212,31017
2020-05-26 16:00:00,4,0,12,62,273,1109,3259,8562,12980,5415,31676
2020-06-03 15:00:00,5,0,15,63,279,1133,3307,8677,13233,5641,32354
2020-06-09 15:00:00,4,0,15,64,282,1143,3342,8760,13427,5788,32825
2020-06-16 11:00:00,4,0,15,65,286,1159,3367,8830,13588,5895,33209
2020-06-23 11:00:00,4,0,16,66,292,1170,3397,8879,13718,6000,33542
2020-06-30 11:00:00,4,0,16,66,296,1174,3411,8909,13792,6068,33736


La tabella sotto mostra le date in cui il numero di nuovi decessi dall'ultimo rilevamento sono negativi per almeno una fascia d'età. In un caso (ultima riga), si riporta che un numero negativo di nuovi decessi in totale.

In [3]:
def highlight_negatives(val):
    if val < 0:
        return 'background-color: red; color: white;'

def show_decreasing_values(df):
    deltas = df.drop(columns='unknown') \
               .diff(1).dropna() \
               .astype(int)
    has_miracles = (deltas < 0).any(axis=1)
    with_negative_deltas = deltas[has_miracles].copy()
    with_negative_deltas['Totale'] = with_negative_deltas.sum(1)
    return with_negative_deltas.style.applymap(highlight_negatives)


show_decreasing_values(data.deaths).set_caption(
    "Bollettini nei quali il numero di nuovi decessi (rispetto al bollettino precedente) è negativo.")

age_group,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,>=90,Totale
date,,,,,,,,,,,
2020-05-26 16:00:00,0,0,-2,1,5,8,40,115,289,203,659
2020-06-09 15:00:00,-1,0,0,1,3,10,35,83,194,147,472
2020-07-21 11:00:00,0,0,0,0,3,-2,2,21,15,21,60
2020-08-25 14:00:00,0,0,0,1,-1,12,12,67,111,66,268
2020-09-08 11:00:00,0,0,-1,0,-2,-4,-25,-61,-179,-117,-389
2020-10-27 11:00:00,0,0,-1,3,5,26,68,134,345,183,763
2020-11-07 11:00:00,0,0,-1,7,24,74,242,657,1195,607,2805


Non conosco la ragione di tali inconsistenze, ma posso presumere che alcuni dei decessi inizialmente attribuiti al coronavirus sono poi stati imputati ad altra causa.

## Correzione dei dati

### Algoritmo di correzione della monoticità

Nel correggere i dati, assumerò che i dati siano tanto più affidabili quanto più recenti. Ciò significa che andrò a considerare "invalidi" i valori all'interno di una serie che sono maggiori di almeno uno dei valori successivi.


1. Per ogni serie numerica, settare a `NaN` tutti i valori all'interno della serie che sono maggiori di almeno un valore successivo.
2. Interpolare i valori "nullificati" al punto uno; io ho usato il metodo di interpolazione PCHIP.
3. Affinché la somma dei valori disaggregati per sesso siano minori o uguali dei corrispondenti totali, 
   calcolare la serie dei totali come massimo tra la serie dei casi totali corretta (al punto 2) e la somma delle serie
   relative ai due sessi (corrette).

### Visualizzazione delle correzioni

In [4]:
def show_corrections(data, variable='deaths', age_group=5, period=None):
    period = slice(*period) if period else slice(data.index[0], data.index[-1])
    
    if isinstance(age_group, int):
        age_group = data.columns.unique(1)[8]
        
    nullified = ic.processing.nullify_local_bumps(data)
    fixed = ic.fix_monotonicity(data)

    (data[variable]
        .loc[period, age_group]
        .plot(style='--X', color='orangered', label='Originale', figsize=(13, 7)))

    (fixed[variable]
        .loc[period, age_group]
        .plot(style='--o', color='green', label='Correzione'))

    (nullified[variable]
        .loc[period, age_group]
        .plot(style='-o', linewidth=2, color='darkblue', label='Tratto comune'))
    
    var_label = 'casi' if variable == 'cases' else 'deceduti'
    plt.title(f'Esempio di correzione della monotonicità [{var_label}, {age_group if age_group != "unknown" else "?"} anni]')
    plt.legend()
    plt.xlabel('')


util.with_interaction(
    show_corrections,
    controls=dict(
        data      = widgets.fixed(data),
        variable  = util.variable_form_field(value='deaths'),
        age_group = ('Età', widgets.Dropdown(value='80-89', options=data.columns.unique(1))), 
        period    = util.period_form_field(data.index, value=('2020-06', None))
    )
)

In [5]:
d = ic.fix_monotonicity(data).deaths
d['Totale'] = d.sum(1)
d.style.background_gradient()

age_group,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,>=90,unknown,Totale
date,,,,,,,,,,,,
2020-03-12 16:00:00,0,0,0,1,1,14,65,274,355,75,18,803
2020-03-16 16:00:00,0,0,0,4,9,46,144,602,727,165,0,1697
2020-03-19 16:00:00,0,0,0,9,25,83,312,1090,1243,285,0,3047
2020-03-23 16:00:00,0,0,0,12,41,168,541,1768,2023,465,1,5019
2020-03-26 16:00:00,0,0,0,17,67,243,761,2403,2702,608,0,6801
2020-03-30 16:00:00,0,0,2,20,89,369,1162,3456,3984,939,5,10026
2020-04-02 16:00:00,0,0,6,29,110,479,1448,4196,5029,1251,2,12550
2020-04-06 16:00:00,1,0,7,34,136,567,1724,4909,5956,1525,1,14860
2020-04-09 16:00:00,1,0,7,36,153,638,1957,5366,6711,1784,1,16654


## Verifiche

In [5]:
fixed = ic.fix_monotonicity(data)

In [6]:
ic.checks.is_non_decreasing(fixed)

True

In [7]:
ic.checks.totals_not_less_than_sum_of_sexes(fixed, 'cases')

True

In [8]:
ic.checks.totals_not_less_than_sum_of_sexes(fixed, 'deaths')

True

## Comparazione con i dati della protezione civile

In [9]:
from pathlib import Path
import requests


def download(url, dirpath='.', fname=None):
    fname = fname or Path(url).name
    path = Path(dirpath, fname)
    resp = requests.get(url)
    with open(path, 'wb') as f:
        f.write(resp.content)
    return path

dpc_url = 'https://github.com/pcm-dpc/COVID-19/raw/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv'
dpc_path = download(dpc_url, dirpath='data')

In [10]:
iss = ic.resample(ic.get(), hour=18)
iss_fixed = ic.fix_monotonicity(iss)
dpc = pd.read_csv(dpc_path, parse_dates=['data'], index_col='data')
dpc.tail()

,stato,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
data,,,,,,,,,,,,,,,,
2020-11-21 17:00:00,ITA,34063,3758,37821,753925,791746,14570,34767,539524,49261,844177.0,536354.0,1380531,20199829,12120989.0,NaN
2020-11-22 17:00:00,ITA,34279,3801,38080,767867,805947,14201,28337,553098,49823,858957.0,549911.0,1408868,20388576,12225850.0,NaN
2020-11-23 17:00:00,ITA,34697,3810,38507,758342,796849,-9098,22930,584493,50453,870461.0,561334.0,1431795,20537521,12303705.0,NaN
2020-11-24 17:00:00,ITA,34577,3816,38393,759993,798386,1537,23232,605330,51306,882238.0,572784.0,1455022,20726180,12398952.0,NaN
2020-11-25 17:00:00,ITA,34313,3848,38161,753536,791697,-6689,25853,637149,52028,896155.0,584719.0,1480874,20956187,12513129.0,NaN


In [12]:

def compare_with_dpc_data(variable='cases', period=None):
    min_start = iss.index[0].strftime('%Y-%m-%d')
    if period is None:
        period = slice(min_start, None)
    elif isinstance(period, tuple):
        period = slice(*period)
    elif not isinstance(slice):
        raise TypeError('period')
    
    if variable == 'cases':
        dpc_data = dpc.loc[period, 'totale_casi']
    else:
        dpc_data = dpc.loc[period, 'deceduti']
    
    dpc_data.plot(figsize=(13,8), label='Dati Protezione Civile', color='blue')
    iss.loc[period, variable].sum(1).plot(label='Dati ISS originali', color='red')
    iss_fixed.loc[period, variable].sum(1).plot(label='Dati ISS corretti', color='green')
    label = 'Casi' if variable == 'cases' else 'deaths'
    plt.title(f"{variable} dall'inizio della pandemia")
    plt.xlabel('')
    util.legend()
    
    
util.with_interaction(
    compare_with_dpc_data,
    controls=dict(
        variable = util.variable_form_field('deaths'),
        period = util.period_form_field(iss.index)
    )
)
